In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import subprocess, os, glob

from scipy.fftpack import fftn, fftfreq, fftshift
from scipy.stats import binned_statistic

# initial thermal spectrum

def pkphi(k, T_initial=4.0, lambdaPRS=1.0, scalefactor=1.0):
    T = T_initial / scalefactor
    m_eff_squared = lambdaPRS * (T**2 / 3 - 1)
    omega = np.sqrt(k**2 + scalefactor**2*m_eff_squared);
    bose = 1 / (np.exp(omega / T) - 1);
    return bose / omega

def pkphidot(k, T_initial=4.0, lambdaPRS=1.0, scalefactor=1.0):
    T = T_initial / scalefactor
    m_eff_squared = lambdaPRS * (T**2 / 3 - 1)
    omega = np.sqrt(k**2 + scalefactor**2*m_eff_squared);
    bose = 1 / (np.exp(omega / T) - 1);
    return bose * omega


# power spectrum estimator:

def power_spectrum2D(f,N,L):
    y = fftn(f)
    P = np.abs(y)**2
    kfreq = fftfreq(N)*N

    kfreq2D = np.meshgrid(kfreq, kfreq)
    K = np.sqrt(kfreq2D[0]**2 + kfreq2D[1]**2)
    K = K.flatten()
    P = P.flatten()
    kbins = np.arange(0.5, N//2, 1.)
    kvals = 0.5 * (kbins[1:] + kbins[:-1])
    # Pk, a, b = binned_statistic(K, P, statistic="mean", bins=kbins)
    Pk, bin_edges, _ = binned_statistic(K, P, statistic = "sum", bins = kbins)
    count, _, _ = binned_statistic(K, P, statistic = "count", bins = kbins)
    for i in range(len(Pk)):
        avg_bin = 0.5 * (bin_edges[i] + bin_edges[i+1])
        Pk[i] /= 2*np.pi*avg_bin
    # Pk *= np.pi * (kbins[1:]**2 - kbins[:-1]**2)

    return kvals*2*np.pi/L, Pk, count

def power_spectrum3D(f,N,L):
    y = fftn(f)
    P = np.abs(y)**2
    kfreq = fftfreq(N)*N

    kfreq3D = np.meshgrid(kfreq, kfreq, kfreq)
    K = np.sqrt(kfreq3D[0]**2 + kfreq3D[1]**2 + kfreq3D[2]**2)
    K = K.flatten()
    P = P.flatten()
    kbins = np.arange(0.5, N//2, 1.)
    kvals = 0.5 * (kbins[1:] + kbins[:-1])
    # Pk, a, b = binned_statistic(K, P, statistic = "mean", bins = kbins)
    Pk, bin_edges, _ = binned_statistic(K, P, statistic = "sum", bins = kbins)
    count, _, _ = binned_statistic(K, P, statistic = "count", bins = kbins)
    for i in range(len(Pk)):
        avg_bin = 0.5 * (bin_edges[i] + bin_edges[i+1])
        Pk[i] /= 4*np.pi*avg_bin**2
    # Pk *= 4/3 * np.pi * (kbins[1:]**3 - kbins[:-1]**3)

    return kvals*(2*np.pi/L), Pk, count


# helper functions for plotting:

def plot_settings(figsize=(7,5)):
    ''' For nicer plots.
    '''
    font = {'size'   : 20, 'family':'STIXGeneral'}
    axislabelfontsize='large'
    matplotlib.rc('font', **font)
    matplotlib.mathtext.rcParams['legend.fontsize']='small'
    matplotlib.rcParams['mathtext.fontset'] = 'cm'
    plt.rcParams['figure.figsize'] = figsize

def plot_field_slice(data, count_id, tau, make_movie=False):
    
    plt.imshow(data, cmap=matplotlib.cm.viridis, interpolation='none')
    plt.xlabel(r"Comoving distance $x/f_a$")
    plt.title(fr" $\tau/\tau_0 = {round(tau, 2)}$", loc = 'left')
    plt.colorbar(fraction=0.05, pad=0.04, shrink=0.75)

    if make_movie:
        plt.savefig("tmp_img_%02d.png" % count_id, facecolor='white')
        plt.clf()
    else:
        plt.show()
    
def plot_axion_slice(data, count_id, tau, make_movie=False):

    plt.imshow(data, cmap=matplotlib.cm.twilight, interpolation='none')
    plt.xlabel(r"Comoving distance $x/f_a$")
    plt.title(fr" $\tau/\tau_0 = {round(tau, 2)}$", loc = 'left')
    cbar = plt.colorbar(fraction=0.05, pad=0.04, shrink=0.75)
    cbar.set_ticks([np.pi-0.001, np.pi/2, 0, -np.pi/2, -np.pi+0.001])
    cbar.set_ticklabels([r"$\pi$", r"$\pi/2$","0",r"$-\pi/2$", r"$-\pi$"])
    cbar.set_label(r"$a(x)/f_a$", rotation=0, labelpad=15)
    
    if make_movie:
        plt.savefig("tmp_img_%02d.png" % count_id, facecolor='white')
        plt.clf()
    else:
        plt.show()

def plot_power_spectrum(data, count_id, tau, make_movie=False):
    ks, pks, count = data

    plt.loglog(ks, pks, "-")
    plt.xlabel(r"$k$")
    plt.ylabel(r"$P\,(k)$", rotation=0, labelpad=30)
    
    if make_movie:
        plt.savefig("tmp_img_%02d.png" % count_id, facecolor='white')
        plt.clf()
    else:
        plt.show()

def convert_format2D(arr, N):
    ret = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            ret[i,j] = arr[i + N * j]
    
    return ret

def read_snapshots2D(N, func=lambda p1, p2: np.arctan2(p1, p2)):
    ''' Note: Appends axion field to snapshots by default.
    '''
    snapshots = []
    count = 0
    done = False
    while not done:
        try:
            phi1 = convert_format2D(np.fromfile(f"snapshot{count}-phi1", dtype=np.float64), N)
            phi2 = convert_format2D(np.fromfile(f"snapshot{count}-phi2", dtype=np.float64), N)
            snapshots.append(func(phi1, phi2))
            count += 1
        except:
            done = True
    
    return snapshots

def plot_snapshots2D(N, func=lambda p1, p2: np.arctan2(p1, p2), plotfunc=plot_axion_slice, **plotfunc_kwargs):
    ''' Returns the plot count.
        Note: Plots axion field by default.
    '''
    timings = pd.read_csv("snapshot-timings.csv")
    tau_vals = np.array(timings['tau'].tolist())

    count = 0
    done = False
    while not done:
        try:
            phi1 = convert_format2D(np.fromfile(f"snapshot{count}-phi1", dtype=np.float64), N)
            phi2 = convert_format2D(np.fromfile(f"snapshot{count}-phi2", dtype=np.float64), N)
            plotfunc(func(phi1, phi2), count, tau_vals[count], **plotfunc_kwargs)
            count += 1
        except:
            done = True

    return count

def convert_format3D(arr, N):
    ret = np.zeros((N, N, N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                ret[i,j,k] = arr[(i * N + j) * N + k]

    return ret

def read_snapshots3D(N, func=lambda p1, p2: np.arctan2(p1, p2)):
    ''' Note: Appends axion field to snapshots by default.
    '''
    snapshots = []
    count = 0
    done = False
    while not done:
        try:
            phi1 = convert_format3D(np.fromfile(f"snapshot{count}-phi1", dtype=np.float64), N)
            phi2 = convert_format3D(np.fromfile(f"snapshot{count}-phi2", dtype=np.float64), N)
            snapshots.append(func(phi1, phi2))
            count += 1
        except:
            done = True
    
    return snapshots

def plot_snapshots3D(N, slice_index, func=lambda p1, p2: np.arctan2(p1, p2), plotfunc=plot_axion_slice, **plotfunc_kwargs):
    ''' Returns the plot count.
        Note: Plots axion field by default.
    '''
    timings = pd.read_csv("snapshot-timings.csv")
    tau_vals = np.array(timings['tau'].tolist())

    count = 0
    done = False
    while not done:
        try:
            phi1 = convert_format3D(np.fromfile(f"snapshot{count}-phi1", dtype=np.float64), N)
            phi2 = convert_format3D(np.fromfile(f"snapshot{count}-phi2", dtype=np.float64), N)
            plotfunc(func(phi1, phi2)[slice_index], count, tau_vals[count], **plotfunc_kwargs)
            count += 1
        except:
            done = True

    return count

def run_ffmpeg(filename, framerate=5):

    subprocess.call(['ffmpeg', '-framerate', str(framerate), '-i', 'tmp_img_%02d.png', '-pix_fmt', 'yuv420p', filename])

    # cleanup image files:
    for fname in glob.glob("tmp_img*"):
        os.remove(fname)


In [ ]:
# Show 2D snapshots:
%matplotlib inline
plot_settings(figsize=(13,12))

make_movie = False

# snapshots = read_snapshots2D(256, func=lambda p1, p2: p1)
snapshots = read_snapshots2D(256)
timings = pd.read_csv("snapshot-timings.csv")
tau = np.array(timings['tau'].tolist())

for i in range(len(snapshots)):
#     plot_field_slice(snapshots[i], tau[i], make_movie=make_movie)
    plot_axion_slice(snapshots[i], tau[i], make_movie=make_movie)

if make_movie:
    framerate = 5
    run_ffmpeg("video.mp4", framerate=framerate)


In [ ]:
# Show 3D snapshots as 2D slices:
%matplotlib inline
plot_settings(figsize=(13,12))

make_movie = False

# snapshots = read_snapshots3D(128, func=lambda p1, p2: p1)
snapshots = read_snapshots3D(128)
timings = pd.read_csv("snapshot-timings.csv")
tau = np.array(timings['tau'].tolist())

for i in range(len(snapshots)):
#     plot_field_slice(snapshots[i][128//2], tau[i], make_movie=make_movie)
    plot_axion_slice(snapshots[i][128//2], tau[i], make_movie=make_movie)

if make_movie:
    framerate = 5
    run_ffmpeg("video.mp4", framerate=framerate)

In [ ]:
# Read single snapshot:
%matplotlib inline
plot_settings(figsize=(13,12))

p1 = convert_format3D(np.fromfile("file-name", dtype=np.float32), 256)
p2 = convert_format3D(np.fromfile("file-name", dtype=np.float32), 256)

ax = np.arctan2(p1,p2)
sx = np.sqrt(p1**2 + p2**2)

plt.imshow(ax[80], cmap=matplotlib.cm.twilight, interpolation='none')
plt.title(r"Axion field $a(x)/f_a$")
locx, _ = plt.xticks([])
locy, _ = plt.yticks([])
cbar = plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_ticks([np.pi-0.001, np.pi/2, 0, -np.pi/2, -np.pi+0.001])
cbar.set_ticklabels([r"$\pi$", r"$\pi/2$","0",r"$-\pi/2$", r"$-\pi$"])
plt.show()

In [ ]:
# Time-series observables:
%matplotlib inline
plot_settings()

ret = pd.read_csv("time-series.csv")
t = np.array(ret['time'].tolist())
tau = a = np.array(ret['scalefactor'].tolist())
xi = np.array(ret['xi'].tolist())

plt.plot(tau, xi, "-")
plt.xlabel(r"$\tau/\tau_0$")
plt.ylabel(r"$\xi$", rotation=0, labelpad=20)
plt.show()

if False:
    # need to divide by a when fields are rescaled:
    phi1_bar = np.array(ret['phi1_bar'].tolist()) / a
    phi2_bar = np.array(ret['phi2_bar'].tolist()) / a
    phidot1_bar = np.array(ret['phidot1_bar'].tolist())
    phidot2_bar = np.array(ret['phidot2_bar'].tolist())
    axion_bar = np.array(ret['axion_bar'].tolist())
    saxion_bar = np.array(ret['saxion_bar'].tolist()) / a

    plt.scatter(phi1_bar, phi2_bar, marker=".", c=t, cmap=matplotlib.cm.viridis)
    plt.colorbar(fraction=0.05, pad=0.04, shrink=0.75)
    plt.xlabel(r"$\langle\phi_1\rangle$")
    plt.ylabel(r"$\langle\phi_2\rangle$", rotation=0, labelpad=30)
    plt.show()

    plt.plot(t, phi1_bar, ".-")
    plt.xlabel(r"$t/f_a$")
    plt.ylabel(r"$\langle \phi_1(x)\rangle$", rotation=0, labelpad=30)
    plt.show()

    plt.plot(t, saxion_bar, ".-")
    plt.xlabel(r"$t/f_a$")
    plt.ylabel(r"$\langle s(x)\rangle$", rotation=0, labelpad=30)
    plt.show()

    plt.plot(t, axion_bar, ".-")
    plt.xlabel(r"$t/f_a$")
    plt.ylabel(r"$\langle a(x)\rangle$", rotation=0, labelpad=30)
    plt.show()

In [ ]:
# Scaling law:
%matplotlib inline
plot_settings((10,6))

colors = ['seagreen','brown','cornflowerblue']
acc = ['03', '015', '0075']
for i in range(3):
    for j in range(4):

        ret = pd.read_csv(f"3DN512_{acc[i]}F_ACC/3DN512_{j}_output_files/time-series.csv")
        t = np.array(ret['time'].tolist())
        tau = a = np.array(ret['scalefactor'].tolist())
        xi = np.array(ret['xi'].tolist())

        plt.plot(tau, xi, "-", color=colors[i])

plt.xlabel(r"$\tau/\tau_0$")
plt.ylabel(r"$\xi$", rotation=0, labelpad=20)
plt.ylim((1, 2))
plt.show()

In [ ]:
# Plot string snapshot:
%matplotlib notebook
plot_settings(figsize=(8,8))

from mpl_toolkits.mplot3d import Axes3D

ret = pd.read_csv("snapshot2-strings.csv", header=None)
xs = ret[0].tolist()
ys = ret[1].tolist()
zs = ret[2].tolist()

ax = plt.axes(projection='3d')
ax.scatter3D(xs, ys, zs, c=zs, cmap=matplotlib.cm.viridis);
plt.show()

In [ ]:
# Power spectrum sanity check:
%matplotlib inline
plot_settings()

dim = 2
N = 256
L = N * 1.0
if dim == 2:
    phi = convert_format2D(np.fromfile("snapshot0-phi1", dtype=np.float64), N)
    ks, pks, count = power_spectrum2D(phi, N, L)
else:
    phi = convert_format3D(np.fromfile("snapshot0-phi1", dtype=np.float64), N)
    ks, pks, count = power_spectrum3D(phi, N, L)

plt.loglog(ks, pks, "-")
# plt.loglog(ks, pkphi(ks, lambdaPRS=0.0)*(2*np.pi/L)**dim)
plt.loglog(ks, pkphi(ks, T_initial=10.0, lambdaPRS=1.0))
plt.xlabel(r"$k$")
plt.ylabel(r"$P\,(k)$", rotation=0, labelpad=30)
# plt.grid(which='both')
plt.show()

plt.loglog(ks, count, ".")
plt.xlabel(r"$k$")
plt.ylabel("Number of modes sampled")
plt.show()

In [ ]:
# Test gaussianity with no interactions:
%matplotlib inline
plot_settings((8, 8))

dim = 2
N = 512
L = N * 1.0

if dim == 2:
    snapshots = read_snapshots2D(N, func=lambda p1, p2: p1) # Just pick out phi1 field for now.
else:
    snapshots = read_snapshots3D(N, func=lambda p1, p2: p1)

timings = pd.read_csv("snapshot-timings.csv")
scalefactor = tau = np.array(timings['tau'].tolist())

magic_const = 1.0
for i in range(len(snapshots)):
    ks, pks, _ = power_spectrum2D(snapshots[i], N, L) if dim == 2 else power_spectrum3D(snapshots[i], N, L)
    plt.loglog(ks, pks, ".", label=fr"$\tau = {round(tau[i],2)}$")
    plt.loglog(ks, pkphi(ks, lambdaPRS=0.0)*magic_const/tau[i]**2)
plt.loglog(ks, pkphi(ks, lambdaPRS=0.0), label=r"$P(k)=n_k/\omega_k$")
plt.ylim((5e-6, 1e-2))
plt.xlabel(r"$k$")
plt.ylabel(fr"$P(k)$")
plt.title(fr" $\tau/\tau_0 = {round(tau[i], 2)}$", loc = 'left')
plt.legend()
plt.show()

ks, pks, _ = power_spectrum2D(snapshots[0], N, L) if dim == 2 else power_spectrum3D(snapshots[0], N, L)
# plt.loglog(ks, pks/(2*np.pi/L)**dim, "-", label=fr"$\tau/\tau_0 = {round(tau[0], 2)}$")
plt.loglog(ks, pks, "-", label=fr"$\tau/\tau_0 = {round(tau[0], 2)}$")
ks, pks, _ = power_spectrum2D(snapshots[-1], N, L) if dim == 2 else power_spectrum3D(snapshots[-1], N, L)
# plt.loglog(ks, pks/(2*np.pi/L)**dim, "-", label=fr"$\tau/\tau_0 = {round(tau[-1], 2)}$")
plt.loglog(ks, pks, "-", label=fr"$\tau/\tau_0 = {round(tau[-1], 2)}$")
plt.loglog(ks, pkphi(ks, lambdaPRS=0.0), "--", label=r"$P\,(k)=n_k/\omega_k$")
plt.loglog(ks, pkphi(ks, lambdaPRS=0.0) / tau[-1]**2, "--", label=r"$P\,(k)=\tau^{-2}n_k/\omega_k$")
plt.xlabel(r"$k$")
plt.ylabel(fr"$P\,(k)$", rotation=0, labelpad=23)
plt.legend()
plt.show()


In [ ]:
# Trying to make sense of hubble drag term to linear order.

plot_settings()

from scipy.interpolate import interp1d
import scipy.integrate as integrate

def compute_field1():

    # y = [phi, phi_prime], t = tau = a
    def dydt(y, t):
        phi, phi_prime = y
        return [phi_prime, -2.0 / t * phi_prime - 2 * phi]
    
    # initial conditions:
    # phi(tau) = 1/tau
    # phi'(tau) = -1/tau^2 = -1
    y0 = [1, 0]
    
    # "timesteps" for scale factor:
    t = np.linspace(1.0, 300.0, 100000)
    
    sol = integrate.odeint(dydt, y0, t)
    return interp1d(t, sol[:, 0]), interp1d(t, sol[:, 1])

def compute_field2():

    # y = [phi, phi_prime], t = tau = a
    def dydt(y, t):
        phi, phi_prime = y
        return [phi_prime, -2.0 / t * phi_prime - phi]
    
    # initial conditions:
    # phi(tau) = 1/tau
    # phi'(tau) = -1/tau^2 = -1
    y0 = [1, 0]
    
    # "timesteps" for scale factor:
    t = np.linspace(1.0, 300.0, 100000)
    
    sol = integrate.odeint(dydt, y0, t)
    return interp1d(t, sol[:, 0]), interp1d(t, sol[:, 1])

phi_a, _ = compute_field1()
phi_b, _ = compute_field2()

t = np.linspace(1.0, 130.0, 1000)

plt.plot(t, phi_a(t))
plt.plot(t, phi_b(t))
plt.show()

print("tau =", tau[-1])
print(phi(tau[0])/phi(tau[-1]))
print(phi(tau[-1]))

